# 핵심 키워드 추출 (Keyword Extraction)

In [1]:
import konlpy

In [2]:
from konlpy.tag import Mecab

In [3]:
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic')
plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings('ignore')

# 1.1 TF-IDF 활용 핵심키워드 추출

## 실습 1. sklearn 활용

In [4]:
import requests
from bs4 import BeautifulSoup

def get_news_by_url(url):
    res = requests.get(url, headers = {
        'User-Agent': 'Mozilla/5.0'
    })
    bs = BeautifulSoup(res.content, 'html.parser')

    title = bs.select_one('#title_area').text #제목
    content = bs.select_one('#contents').get_text().replace('\n', " ") #본문
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    return  content.strip()

docs = []
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=001&aid=0011614790') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=102&oid=014&aid=0004424362') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=119&aid=0002402191') )
docs.append( get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=030&aid=0002882728') )
len(docs)

5

### 1) 전처리

In [5]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
    # 명사와 동사만으로 문서 전처리
    preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]


'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

count_vectorizer = CountVectorizer(max_df=0.85, max_features=10000)
word_count_vector = count_vectorizer.fit_transform(preprocessed_docs)
list(count_vectorizer.vocabulary_.keys())[:10]

['과기', '정통부', '유영민', '장관', '참석', '기념행사', '투입', '여종', '구축', '민간']

In [7]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer()

### 3) 핵심키워드 추출

In [8]:
def sort_keywords(keywords):
    return sorted(zip(keywords.col, keywords.data), key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [9]:
doc = preprocessed_docs[0] # 핵심키워드 추출할 문서 조회

feature_names = count_vectorizer.get_feature_names() # TF-IDF 단어 목록
tf_idf_vector = tfidf_transformer.transform(count_vectorizer.transform([doc])) # 문서의 tf-idf 추출
sorted_keywords = sort_keywords(tf_idf_vector.tocoo()) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(feature_names, sorted_keywords, 5)

print("\n===== 원문 =====")
print(docs[0][:100])
print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


===== 원문 =====
과기정통부, 22일 유영민 장관 등 참석해 기념행사2021년까지 1516억원 투입, 5100여종 데이터 구축민간 클라우드 통한 외부연계체계도.."개방성 강화"[이데일리 이재운 기자

=== 핵심키워드 ===
('플랫', 0.2501844954610449)
('계획', 0.21444385325232418)
('정통부', 0.1787032110436035)
('데일리', 0.1787032110436035)
('과기', 0.1787032110436035)


In [10]:
tf_idf_vector.tocoo().data

array([0.03574861, 0.03574861, 0.03574861, 0.03574861, 0.05768351,
       0.07149723, 0.03574861, 0.03574861, 0.03574861, 0.02884176,
       0.03574861, 0.14420878, 0.03574861, 0.03574861, 0.02884176,
       0.02884176, 0.07149723, 0.11536703, 0.03574861, 0.03574861,
       0.04788253, 0.03574861, 0.03574861, 0.25024029, 0.03574861,
       0.02884176, 0.03574861, 0.02884176, 0.02884176, 0.03574861,
       0.03574861, 0.10724584, 0.17305054, 0.07149723, 0.03574861,
       0.10724584, 0.03574861, 0.07149723, 0.03574861, 0.03574861,
       0.02884176, 0.02884176, 0.07149723, 0.03574861, 0.04028029,
       0.02394126, 0.07149723, 0.03574861, 0.03574861, 0.10724584,
       0.03574861, 0.03574861, 0.10724584, 0.03574861, 0.03574861,
       0.02884176, 0.03574861, 0.03574861, 0.02394126, 0.03574861,
       0.02884176, 0.03574861, 0.03574861, 0.03574861, 0.10724584,
       0.02884176, 0.02884176, 0.02884176, 0.03574861, 0.11970632,
       0.07149723, 0.17874307, 0.02884176, 0.04028029, 0.02884


---


## 실습 2. gensim 활용


### 1) 전처리

In [11]:
from konlpy.tag import Mecab
mecab = Mecab()

preprocessed_docs = []
for doc in docs :
    # 명사와 동사만으로 문서 전처리
    preprocessed_docs.append(' '.join([token[0] for token in mecab.pos(doc) if token[1][0] in ['N', 'V']]))
preprocessed_docs[0][:100]

'과기 정통부 일 유영민 장관 등 참석 기념행사 년 억 원 투입 여종 데이터 구축 민간 클라우드 통한 외부 연계 체계 개방 강화 데일리 이재운 기자 국가 차원 빅 데이터 활용 시대 '

### 2) TF-IDF 계산

In [12]:
from gensim.models import TfidfModel
from gensim.corpora import Dictionary

document_ls = [doc.split() for doc in preprocessed_docs]
dct = Dictionary(document_ls) # 인덱스(key) - 단어(value) 인 딕셔너리 생성
corpus = [dct.doc2bow(doc) for doc in document_ls] # 각 문서에 포함된 단어를 인덱스로 변환하여 corpus 생성
tfidf = TfidfModel(corpus) # TF-IDF 산출

### 3) 핵심키워드 추출

In [13]:
def sort_keywords(tfidf):
    return sorted(tfidf, key=lambda x: (x[1], x[0]), reverse=True)

def extract_keywords(feature_names, sorted_keywords, n=5):
    return [(feature_names[idx], score) for idx, score in sorted_keywords[:n]]

In [14]:
doc = corpus[0]

sorted_keywords = sort_keywords(tfidf[doc]) # TF-IDF를 기준으로 역순 정렬

# 사용자가 지정한 갯수만큼 키워드 추출
keywords = extract_keywords(dct, sorted_keywords, 5)

print("\n=== 핵심키워드 ===")
for k in keywords:
    print(k)


=== 핵심키워드 ===
('플랫', 0.25554346587361404)
('폼', 0.25554346587361404)
('계획', 0.21903725646309777)
('정통부', 0.18253104705258147)
('데일리', 0.18253104705258147)


In [15]:
tfidf[doc]

[(0, 0.03650620941051629),
 (1, 0.020783840793288627),
 (2, 0.07301241882103258),
 (3, 0.07301241882103258),
 (6, 0.03650620941051629),
 (7, 0.023173689459390606),
 (8, 0.020783840793288627),
 (9, 0.03650620941051629),
 (10, 0.03650620941051629),
 (11, 0.03650620941051629),
 (13, 0.06235152237986588),
 (15, 0.03650620941051629),
 (16, 0.041567681586577254),
 (17, 0.020783840793288627),
 (18, 0.03650620941051629),
 (19, 0.020783840793288627),
 (20, 0.03650620941051629),
 (21, 0.0050614721760609575),
 (22, 0.03650620941051629),
 (23, 0.020783840793288627),
 (24, 0.0050614721760609575),
 (25, 0.07301241882103258),
 (26, 0.02530736088030479),
 (27, 0.03650620941051629),
 (28, 0.21903725646309777),
 (29, 0.03650620941051629),
 (30, 0.010122944352121915),
 (31, 0.03650620941051629),
 (32, 0.07301241882103258),
 (33, 0.03650620941051629),
 (34, 0.011586844729695303),
 (35, 0.03650620941051629),
 (36, 0.18253104705258147),
 (37, 0.041567681586577254),
 (38, 0.03650620941051629),
 (39, 0.041567



---



# 1.2 Textrank
https://web.eecs.umich.edu/~mihalcea/papers/mihalcea.emnlp04.pdf

<img src="https://3.bp.blogspot.com/-yp0Lr3ec5EY/XIs6znCcO_I/AAAAAAAAAPY/xtZxe_OYtH0xeuWsp4Qd4DQrunGMpVQmQCLcBGAs/s640/keyword-extraction-textrank.png" />

## 실습 1. 행렬 활용


In [11]:
doc = '딸기 바나나 사과 딸기 파인애플'

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [12]:
tokens = doc.split()

In [13]:
tokens

['딸기', '바나나', '사과', '딸기', '파인애플']

### 2) Unique한 토큰 목록 생성

그래프 생성을 위해서 Unique한 토큰 목록 생성

In [14]:
unique_tokens = []
for token in tokens:
    have_token = token in unique_tokens
    if have_token:
        pass
    else:
        unique_tokens.append(token)

unique_tokens 

['딸기', '바나나', '사과', '파인애플']

In [15]:
nodes = unique_tokens
node_scores = [1.0]*len(nodes)

In [16]:
nodes, node_scores

(['딸기', '바나나', '사과', '파인애플'], [1.0, 1.0, 1.0, 1.0])

In [17]:
node2idx = {node: i for i, node in enumerate(nodes)}

In [18]:
node2idx

{'딸기': 0, '바나나': 1, '사과': 2, '파인애플': 3}

In [21]:
node_scores[ node2idx['사과'] ]

1.0

### 3) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex)
*   weighted_edge 행렬은 노드간 가중치 정보를 담고 있음
*   weighted_edge[i][j] 는 i번째 단어와 j번째 단어의 가중치를 의미
*   weighted_edge[i][j] 가 0인 경우는 노드간 연결이 없음을 의미
*   모든 노드는 1로 초기화

In [30]:
edges = {}

window_size = 3

for i in range(len(tokens)-window_size+1):
    context = tokens[i:i+window_size]
    print(context)
    for s_token in range(window_size-1):
        for e_token in range(s_token+1, window_size):
            print('s', context[s_token])
            print('e', context[e_token])

['딸기', '바나나', '사과']
s 딸기
e 바나나
s 딸기
e 사과
s 바나나
e 사과
['바나나', '사과', '딸기']
s 바나나
e 사과
s 바나나
e 딸기
s 사과
e 딸기
['사과', '딸기', '파인애플']
s 사과
e 딸기
s 사과
e 파인애플
s 딸기
e 파인애플


### 4) 각 노드의 score계산
각 노드와 연결된 weighted edge의 값을 합산

### 5) 핵심 단어 추출

---

## 실습 2. 그래프 활용

In [ ]:
import requests
from bs4 import BeautifulSoup

def get_news_by_url(url):
    res = requests.get(url, headers = {
        'User-Agent': 'Mozilla/5.0'
    })
    bs = BeautifulSoup(res.content, 'html.parser')

    title = bs.select_one('#title_area').text #제목
    content = bs.select_one('#contents').get_text().replace('\n', " ") #본문
    content = content.replace("// flash 오류를 우회하기 위한 함수 추가 function _flash_removeCallback() {}", "")
    return content.strip()

doc = get_news_by_url('https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=105&oid=018&aid=0004430108')
doc[:50]

### 1) 토큰화 (Tokenization)

분석 텍스트 정제

In [ ]:
from konlpy.tag import Mecab
mecab = Mecab()

nodes = [token for token in mecab.pos(doc) if token[1] in ['NNG','NNP']] #NNG, NNP를 스코어 계산 대상(노드)로 제한
tokens = [token for token in mecab.pos(doc)] #탐색할 토큰 전체

### 2) 그래프 생성 (weighted edge 계산)

*   TextRank는 그래프 기반 모델
*   각 단어(토큰)은 그래프의 노드(vertex) 없음을 의미
*   모든 노드는 1로 초기화

In [ ]:
import numpy as np
import math
import networkx as nx

# 윈도내 동시 등장한 토큰으로 그래프를 생성
def connect(nodes, tokens) :
    window_size = 2 # coocurrence를 판단하기 위한 window 사이즈 설정

    edges = []
    for window_start in range(0,(len(tokens)-window_size+1)):
        window = tokens[window_start:window_start+window_size]
        for i in range(window_size) :
            for j in range(i+1, window_size) :
                if (window[i] in nodes) & (window[j] in nodes) : #윈도 내 단어가 노드에 속하는 경우만 엣지로 연결
                    edges.append((window[i], window[j]))
                    print((window[i], window[j]))
    return edges

# tokens = ['딸기', '바나나', '사과', '딸기', '파인애플']
# nodes = ['바나나', '사과', '파인애플', '딸기']

graph=nx.diamond_graph()
graph.clear()
graph.add_nodes_from(list(set(nodes))) #node 등록
graph.add_edges_from(connect(nodes, tokens)) #edge 연결

### 3) 스코어 계산 및 핵심키워드 추출

In [ ]:
scores = nx.pagerank(graph) #pagerank 계산
rank = sorted(scores.items(), key=lambda x: x[1], reverse=True) #score 역순 정렬
print("\n=== 핵심키워드 ===")
rank[:5]

In [ ]:
import networkx as nx
from IPython.core.display import Image
from networkx.drawing.nx_pydot import to_pydot

d = to_pydot(graph)
d.set_dpi(600)
d.set_rankdir("LR")
Image(d.create_png(), width=600)

---

## 실습 3. TextRank 핵심 구 추출

### 1) 불용어를 기준으로 구 추출

In [ ]:
phrases = []
phrase = ' '

for word in tokens:
    if word[1][0] != 'N':
        if phrase != ' ':
            phrases.append(phrase.strip())
        phrase= ' '
    else:
        phrase+=word[0]
        phrase+= ' '

print(phrases)

In [ ]:
unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)

print(unique_phrases)

### 2) 각 구의 Score 계산

앞서 산출한 각 단어별 점수를 합산

In [ ]:
vocabs = dict((r[0][0], r[1]) for r in rank)
phrase_scores = []
keywords = []

for phrase in unique_phrases:
    phrase_score = 0
    for word in phrase.split():
        if word in vocabs.keys():
            phrase_score+=vocabs[word]

    phrase_scores.append((phrase, phrase_score))

for phrase_score in phrase_scores[:10]:
    print(f"keyword: {phrase_score[0]}, Score : {phrase_score[1]}")

### 3) 각 구를 Score로 정렬하여 핵심 구 추출

In [ ]:
sorted_phrase_score = sorted(phrase_scores, key=lambda tup: tup[1], reverse=True)
sorted_phrase_score[:10]



---



## 실습 4. gensim 활용

- summarization.keywords(document, words) : document를 요약하여 words만큼 키워드를 추출

In [22]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from gensim.summarization import keywords
keywords(doc, words=5).split('\n')